Python code to process data from https://www.kaggle.com/kkhandekar/cheapest-electric-cars

In [240]:
import pandas as pd
import numpy as np

Conversion rate for pound to euro

In [241]:
conversionRate = 1.2

Read in data

In [242]:
#Read in csv file
url = 'https://raw.githubusercontent.com/WesterAl/AML-Electric-Cars-project-work-Price-Prediction-/main/Cheapestelectriccars-EVDatabase.csv'
df = pd.read_csv(url)

df.head()

,Name,Subtitle,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,Drive,NumberofSeats,PriceinGermany,PriceinUK
0,Opel Ampera-e,Battery Electric Vehicle | 58 kWh,7.3 sec,150 km/h,335 km,173 Wh/km,210 km/h,Front Wheel Drive,5,"€42,990",NaN
1,Renault Kangoo Maxi ZE 33,Battery Electric Vehicle | 31 kWh,22.4 sec,130 km/h,160 km,194 Wh/km,-,Front Wheel Drive,5,NaN,"£31,680"
2,Nissan Leaf,Battery Electric Vehicle | 36 kWh,7.9 sec,144 km/h,220 km,164 Wh/km,230 km/h,Front Wheel Drive,5,"€29,990","£25,995"
3,Audi e-tron Sportback 55 quattro,Battery Electric Vehicle | 86.5 kWh,5.7 sec,200 km/h,375 km,231 Wh/km,600 km/h,All Wheel Drive,5,NaN,"£79,900"
4,Porsche Taycan Turbo S,Battery Electric Vehicle | 83.7 kWh,2.8 sec,260 km/h,390 km,215 Wh/km,860 km/h,All Wheel Drive,4,"€186,336","£138,830"


##Data processing

Price columns


Price column is same as PriceinGermany column. If PriceinGermany has Nan -> Nan will be updated with value from PriceinUK with a conversion rate

Rows 28, 119 and 128 have no price at all and will be removed. 
(Volkswagen e-Up!, Seres 3, JAC iEV7s)

In [243]:
#Remove £ sign
df['PriceinUK'] = df.PriceinUK.str.split('£',expand=True)[1]
#Remove ,
df['PriceinUK'] =df.PriceinUK.str.replace(',','').astype('float32')
#Remove € sign
df['PriceinGermany'] = df.PriceinGermany.str.split('€',expand=True)[1]
#Remove ,
df['PriceinGermany'] =df.PriceinGermany.str.replace(',','').astype('float32')

#Add column price. A copy of PriceinGermany.
df['price'] = df['PriceinGermany']

#Loop through rows and change values in price column
for a in range(df.shape[0]):
  price = df.loc[a, ['price']][0]
  ukColumn = df.loc[a, ['PriceinUK']][0]
  #If price is Nan and ukColumn is not Nan -> price becomes PriceinUK converted to € with conversionRate variable
  if np.isnan(price) == True and np.isnan(ukColumn) != True:
    price = ukColumn * conversionRate
  #Update value in price column
  df.loc[a, ['price']] = price  

#drop columns PriceinGermany and PriceinUK
df.drop(columns=['PriceinGermany', 'PriceinUK'], inplace=True)

#Remove rows where value in "price" column is Nan
df = df[df['price'].notna()]

df.head()


,Name,Subtitle,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,Drive,NumberofSeats,price
0,Opel Ampera-e,Battery Electric Vehicle | 58 kWh,7.3 sec,150 km/h,335 km,173 Wh/km,210 km/h,Front Wheel Drive,5,42990.0
1,Renault Kangoo Maxi ZE 33,Battery Electric Vehicle | 31 kWh,22.4 sec,130 km/h,160 km,194 Wh/km,-,Front Wheel Drive,5,38016.0
2,Nissan Leaf,Battery Electric Vehicle | 36 kWh,7.9 sec,144 km/h,220 km,164 Wh/km,230 km/h,Front Wheel Drive,5,29990.0
3,Audi e-tron Sportback 55 quattro,Battery Electric Vehicle | 86.5 kWh,5.7 sec,200 km/h,375 km,231 Wh/km,600 km/h,All Wheel Drive,5,95880.0
4,Porsche Taycan Turbo S,Battery Electric Vehicle | 83.7 kWh,2.8 sec,260 km/h,390 km,215 Wh/km,860 km/h,All Wheel Drive,4,186336.0


Extract battery size from "Subtitle" column and drop "Subtitle" column

In [244]:
#Remove "Battery Electric Vehicle | " from 'Subtitle' column and add new column called BatteryCapacity
df['BatteryCapacity'] = df['Subtitle'].apply(lambda x: x.split(' ')[10])

#Drop "Subtitle" column
df.drop(columns=['Subtitle'], inplace=True)

df.head()

,Name,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,Drive,NumberofSeats,price,BatteryCapacity
0,Opel Ampera-e,7.3 sec,150 km/h,335 km,173 Wh/km,210 km/h,Front Wheel Drive,5,42990.0,58
1,Renault Kangoo Maxi ZE 33,22.4 sec,130 km/h,160 km,194 Wh/km,-,Front Wheel Drive,5,38016.0,31
2,Nissan Leaf,7.9 sec,144 km/h,220 km,164 Wh/km,230 km/h,Front Wheel Drive,5,29990.0,36
3,Audi e-tron Sportback 55 quattro,5.7 sec,200 km/h,375 km,231 Wh/km,600 km/h,All Wheel Drive,5,95880.0,86.5
4,Porsche Taycan Turbo S,2.8 sec,260 km/h,390 km,215 Wh/km,860 km/h,All Wheel Drive,4,186336.0,83.7


Remove sec, km/h etc. from columns

In [245]:
df['Acceleration'] = df['Acceleration'].apply(lambda x: x.split(' ')[0])
df['TopSpeed'] = df['TopSpeed'].apply(lambda x: x.split(' ')[0])
df['Range'] = df['Range'].apply(lambda x: x.split(' ')[0])
df['Efficiency'] = df['Efficiency'].apply(lambda x: x.split(' ')[0])

#Remove km/h and change "-" to Nan in column FastChargeSpeed
df['FastChargeSpeed']=pd.to_numeric(df['FastChargeSpeed'].str.split(' ',expand=True)[0],errors='coerce') #"If ‘coerce’, then invalid parsing will be set as NaN." From pandas documantation
#Drop rows with Nan in column FastChargeSpeed
df = df[df['FastChargeSpeed'].notna()]

df.head()

,Name,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,Drive,NumberofSeats,price,BatteryCapacity
0,Opel Ampera-e,7.3,150,335,173,210.0,Front Wheel Drive,5,42990.0,58
2,Nissan Leaf,7.9,144,220,164,230.0,Front Wheel Drive,5,29990.0,36
3,Audi e-tron Sportback 55 quattro,5.7,200,375,231,600.0,All Wheel Drive,5,95880.0,86.5
4,Porsche Taycan Turbo S,2.8,260,390,215,860.0,All Wheel Drive,4,186336.0,83.7
5,Nissan e-NV200 Evalia,14.0,123,165,218,170.0,Front Wheel Drive,7,43433.0,36


Change Drive to 1 if front wheel, 2 if rear wheel and 3 if all wheel

In [246]:
df['Drive'] = df['Drive'].replace('Front Wheel Drive', '1')
df['Drive'] = df['Drive'].replace('Rear Wheel Drive', '2')
df['Drive'] = df['Drive'].replace('All Wheel Drive', '3')

df.head() 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,Name,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,Drive,NumberofSeats,price,BatteryCapacity
0,Opel Ampera-e,7.3,150,335,173,210.0,1,5,42990.0,58
2,Nissan Leaf,7.9,144,220,164,230.0,1,5,29990.0,36
3,Audi e-tron Sportback 55 quattro,5.7,200,375,231,600.0,3,5,95880.0,86.5
4,Porsche Taycan Turbo S,2.8,260,390,215,860.0,3,4,186336.0,83.7
5,Nissan e-NV200 Evalia,14.0,123,165,218,170.0,1,7,43433.0,36


Make sure all values are float or integer. No number should be string

In [247]:
myList = []
for element in df['Acceleration']:
  element = float(element)
  myList.append(element)
df['Acceleration'] = myList

myList = []
for element in df['TopSpeed']:
  element = int(element)
  myList.append(element)
df['TopSpeed'] = myList

myList = []
for element in df['Range']:
  element = int(element)
  myList.append(element)
df['Range'] = myList

myList = []
for element in df['Efficiency']:
  element = int(element)
  myList.append(element)
df['Efficiency'] = myList

myList = []
for element in df['FastChargeSpeed']:
  element = int(element)
  myList.append(element)
df['FastChargeSpeed'] = myList

myList = []
for element in df['NumberofSeats']:
  element = int(element)
  myList.append(element)
df['NumberofSeats'] = myList

myList = []
for element in df['price']:
  element = int(element)
  myList.append(element)
df['price'] = myList

myList = []
for element in df['BatteryCapacity']:
  element = float(element)
  myList.append(element)
df['BatteryCapacity'] = myList

myList = []
for element in df['Drive']:
  element = int(element)
  myList.append(element)
df['Drive'] = myList

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 179
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             172 non-null    object 
 1   Acceleration     172 non-null    float64
 2   TopSpeed         172 non-null    int64  
 3   Range            172 non-null    int64  
 4   Efficiency       172 non-null    int64  
 5   FastChargeSpeed  172 non-null    int64  
 6   Drive            172 non-null    int64  
 7   NumberofSeats    172 non-null    int64  
 8   price            172 non-null    int64  
 9   BatteryCapacity  172 non-null    float64
dtypes: float64(2), int64(7), object(1)
memory usage: 14.8+ KB


Change order of columns


In [248]:
df = df[['Name', 	'Acceleration', 	'TopSpeed', 	'Range', 'BatteryCapacity', 	'Efficiency', 	'FastChargeSpeed', 	'Drive', 	'NumberofSeats', 'price']]

df.head()

,Name,Acceleration,TopSpeed,Range,BatteryCapacity,Efficiency,FastChargeSpeed,Drive,NumberofSeats,price
0,Opel Ampera-e,7.3,150,335,58.0,173,210,1,5,42990
2,Nissan Leaf,7.9,144,220,36.0,164,230,1,5,29990
3,Audi e-tron Sportback 55 quattro,5.7,200,375,86.5,231,600,3,5,95880
4,Porsche Taycan Turbo S,2.8,260,390,83.7,215,860,3,4,186336
5,Nissan e-NV200 Evalia,14.0,123,165,36.0,218,170,1,7,43433


In [249]:
df.shape

(172, 10)

In [250]:
df

,Name,Acceleration,TopSpeed,Range,BatteryCapacity,Efficiency,FastChargeSpeed,Drive,NumberofSeats,price
0,Opel Ampera-e,7.3,150,335,58.0,173,210,1,5,42990
2,Nissan Leaf,7.9,144,220,36.0,164,230,1,5,29990
3,Audi e-tron Sportback 55 quattro,5.7,200,375,86.5,231,600,3,5,95880
4,Porsche Taycan Turbo S,2.8,260,390,83.7,215,860,3,4,186336
5,Nissan e-NV200 Evalia,14.0,123,165,36.0,218,170,1,7,43433
...,...,...,...,...,...,...,...,...,...,...
175,MG Marvel R,7.9,200,340,65.0,191,390,2,5,40000
176,Tesla Model 3 Long Range Dual Motor,4.4,233,490,76.0,155,820,3,5,53560
177,MG MG5 EV Long Range,7.7,185,340,57.0,168,340,1,5,31794
178,Audi Q4 e-tron 45 quattro,6.9,180,385,76.6,199,470,3,5,50900


In [251]:
#Download file to local drive
#from google.colab import files
#df.to_csv('Processed_Cheapestelectriccars-EVDatabase.csv') 
#files.download('Processed_Cheapestelectriccars-EVDatabase.csv')